# Closures
## Capturing Variables
In most languages with closures, garbage collection plays an important role. For example, consider this JavaScript code:
```Rust
// Start an animation that rearranges the rows in a table of cities.
function startSortingAnimation(cities, stat) {
    // Helper function that we'll use to sort the table. 
    // Note that this function refers to stat.
    function keyfn(city) {
        return city.get_statistic(stat); 
    }
    if (pendingSort) pendingSort.cancel();
    // Now kick off an animation, passing keyfn to it. 
    // The sorting algorithm will call keyfn later. 
    pendingSort = new SortingAnimation(cities, keyfn);
}
```
The closure `keyfn` is stored in the new `SortingAnimation` object. It’s meant to be called after `startSortingAnimation` returns. Now, normally when a function returns, all its variables and arguments go out of scope and are discarded. But here, the JavaScript engine must keep `stat` around somehow, since the closure uses it. Most JavaScript engines do this by allocating stat in the heap and letting the garbage collector reclaim it later.

Rust doesn’t have garbage collection. How will this work? To answer this question, we’ll look at two examples.

### Closures That Borrow
```Rust
/// Sort by any of several different statistics.
fn sort_by_statistic(cities: &mut Vec<City>, stat: Statistic) { 
    cities.sort_by_key(|city| -city.get_statistic(stat));
}
```
In this case, when Rust creates the closure, it automatically borrows a reference to stat. It stands to reason: the closure refers to stat, so it must have a reference to it.
The rest is simple. The closure is subject to the rules about borrowing and lifetimes that we described in **Chapter 5**. In particular, since the closure contains a reference to stat, Rust won’t let it outlive stat. Since the closure is only used during sorting, this example is fine.

In short, Rust ensures safety by using lifetimes instead of garbage collection. Rust’s way is faster: even a fast GC allocation will be slower than storing stat on the stack, as Rust does in this case.